<a href="https://colab.research.google.com/github/limshaocong/analyticsEdge/blob/main/Zero_shot_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers==3.1.0

     |████████████████████████████████| 884 kB 5.4 MB/s 
     |████████████████████████████████| 3.0 MB 48.5 MB/s 
     |████████████████████████████████| 1.2 MB 38.3 MB/s 
     |████████████████████████████████| 895 kB 60.8 MB/s 


In [2]:
from transformers import pipeline
import pandas as pd

In [3]:
classifier = pipeline("zero-shot-classification", # transfer learning
                      model = "joeddav/xlm-roberta-large-xnli", # multi-lingual model
                      device = 0) # to utilize GPU

Downloading:   0%|          | 0.00/734 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

In [4]:
import re
from datetime import datetime
import os.path
from os import path
import requests

In [5]:
def analyse(ticker):
  
  file_path = path + ticker + str("_tweets_2020_cleaned.csv")
  
  df = pd.read_csv(file_path)
  
  df['Text_clean'] = df['Text_clean'].astype(str)

  texts = df["Text_clean"]

  chunks = [texts[x : x + 100] for x in range(0, len(texts), 100)] # split into chunks

  results_list = []

  for idx in range(len(chunks)):

    cp1 = datetime.now()
    
    chunk = chunks[idx].tolist()

    candidate_labels = ["positive", "negative", "neutral"]

    hypothesis_template = "The sentiment of this tweet is {}."

    #dicts = classifier(text, candidate_labels, hypothesis_template = hypothesis_template)
    #result = dicts["labels"][0]
    #results_list.append(result)
    
    list_results_dicts = classifier(chunk, candidate_labels, hypothesis_template = hypothesis_template)

    if type(list_results_dicts) != dict:
      new_list = [i["labels"][0] for i in list_results_dicts] # not robust when chunk size == 1
      results_list.extend(new_list)
    
    else:
      new_list = list_results_dicts["labels"][0]
      results_list.append(new_list)
    
    #results_list.extend(new_list)

    cp2 = datetime.now()

    print(str(idx + 1) + str("/") + str(len(chunks)) + str(" --- ") + str(cp2 - cp1))

  return results_list

In [6]:
from google.colab import files

ticker_path = "https://raw.githubusercontent.com/limshaocong/analyticsEdge/main/TwitterScraper/Tickers_reduced_full.csv"
tickers = pd.read_csv(ticker_path)
tickers_list = tickers["Tickers"].tolist()
tickers_list = tickers_list[126:]

path = "https://raw.githubusercontent.com/limshaocong/analyticsEdge/main//Datasets/Tweet_Analysis/"

In [ ]:
tickers_list[:20]

In [15]:
"""
for ticker in tickers_list:
  
  file_path = path + ticker + str("_tweets_2020_cleaned.csv")
  
  response = requests.get(file_path)

  if response.status_code < 400:
 
    print(ticker + str(" is present"))

    file_path = path + ticker + str("_tweets_2020_cleaned.csv")

    print(file_path)
  
    df = pd.read_csv(file_path)

    results_list = analyse(ticker)

    df["Sentiment"] = results_list

    newfilename = ticker + str("_tweets_2020_tagged.csv")

    df.to_csv(newfilename, index = False)
  
  else:

    print(ticker + str(" is missing"))

    skipped.append(ticker)

    print(skipped)

    continue

"""

'\nfor ticker in tickers_list:\n  \n  file_path = path + ticker + str("_tweets_2020_cleaned.csv")\n  \n  response = requests.get(file_path)\n\n  if response.status_code < 400:\n \n    print(ticker + str(" is present"))\n\n    file_path = path + ticker + str("_tweets_2020_cleaned.csv")\n\n    print(file_path)\n  \n    df = pd.read_csv(file_path)\n\n    results_list = analyse(ticker)\n\n    df["Sentiment"] = results_list\n\n    newfilename = ticker + str("_tweets_2020_tagged.csv")\n\n    df.to_csv(newfilename, index = False)\n  \n  else:\n\n    print(ticker + str(" is missing"))\n\n    skipped.append(ticker)\n\n    print(skipped)\n\n    continue\n\n'

In [27]:
skipped = ['CAT',
 'CMCSA',
 'CORE',
 'CRDA',
 'CSCO',
 'CUBI',
 'DISCK',
 'DUK',
 'EBAY',
 'EPR',
 'F',
 'FCNCA',
 'FNB',
 'FOX',
 'FOXA',
 'FWONA',
 'FWONK',
 'MA',
 'MDP',
 'MDT',
 'MMM',
 'MRK',
 'MS',
 'NEE',
 'NKE',
 'NWS',
 'NWSA',
 'OKE',
 'PENN',
 'PFE',
 'PK',
 'POST',
 'PRGO',
 'PRU',
 'QADA',
 'QCOM',
 'QRTEA',
 'RMD',
 'ROG',
 'SBRA',
 'SBUX',
 'SF',
 'SITC',
 'SLM',
 'STAG',
 'STT',
 'STZ',
 'SWM',
 'SYY',
 'T',
 'TAP',
 'TPB',
 'TSLA',
 'TWTR',
 'UA',
 'UAA',
 'AMZN',
 'GOOG',
 'GOOGL',
 'NFLX',
 'IBM',
 'BA',
 'FB']

In [34]:
skipped_1 = skipped[:15]
skipped_2 = skipped[15:30]
skipped_3 = skipped[30:45]
skipped_4 = skipped[45:60]
skipped_5 = skipped[60:] # slow and painful

In [35]:
len(skipped_1)

15

In [28]:
complete = []

for ticker in skipped:
  
  file_path = path + ticker + str("_tweets_2020_cleaned.csv")
  
  response = requests.get(file_path)

  if response.status_code < 400:
      
    print(ticker + str(" is present"))

    file_path = path + ticker + str("_tweets_2020_cleaned.csv")

    print(file_path)
  
    df = pd.read_csv(file_path)

    results_list = analyse(ticker)

    df["Sentiment"] = results_list

    newfilename = ticker + str("_tweets_2020_tagged.csv")

    df.to_csv(newfilename, index = False)

    complete.append(ticker)
  
  else:

    print(ticker + str(" is missing"))

    continue

CAT is present
https://raw.githubusercontent.com/limshaocong/analyticsEdge/main//Datasets/Tweet_Analysis/CAT_tweets_2020_cleaned.csv


KeyboardInterrupt: ignored

In [25]:
skipped = skipped - complete

63